In [59]:
# Import necessary libraries
from azure.storage.blob import BlobServiceClient
import pandas as pd
import io

# Initialize a BlobServiceClient
sas_token = "sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupiytfx&se=2024-05-13T13:01:11Z&st=2024-04-13T05:01:11Z&spr=https,http&sig=X362UUEwAfkGDCBr2G0ZOc2SX%2FLg9LOddGvPHZ%2Fff00%3D"  # Ensure this is URL-encoded
blob_service_client = BlobServiceClient(account_url="https://sadukedatauseprod.blob.core.windows.net", credential=sas_token)

# Function to list blobs in a container
def list_blobs(container_name):
    container_client = blob_service_client.get_container_client(container=container_name)
    return [blob.name for blob in container_client.list_blobs()]

# Function to load data from a specific blob within a container
def load_data(container_name, blob_name):
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    blob_content = blob_client.download_blob().readall()
    return pd.read_csv(io.StringIO(blob_content.decode('utf-8')))

CONTAINER='mimiciii'


In [60]:
# ICD = load_data(CONTAINER, 'DIAGNOSES_ICD.csv')
# Notes=load_data(CONTAINER,"NOTEEVENTS.csv")
# Micro = load_data(CONTAINER, 'MICROBIOLOGYEVENTS.csv')
# Admissions = load_data("mimiciii","ADMISSIONS.csv")
# Patients = load_data(CONTAINER, 'PATIENTS.csv')

# Part 1: ICD Codes
In this section, we will filter all patients who were billed with an ICD9 code indicating sepsis, namely: 99591 and 99592

In [61]:
# Load and Filter ICD Data
print(f"Original columns for ICD: {ICD.columns}\n")
sepsis_ICD = ICD[ICD.ICD9_CODE.isin(['99591', '99592'])]
sepsis_ICD = sepsis_ICD[['HADM_ID', 'SUBJECT_ID']]
sepsis_ICD['Sepsis by ICD'] = True

sepsis_ICD.head(10)

Original columns for ICD: Index(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'SEQ_NUM', 'ICD9_CODE'], dtype='object')



,HADM_ID,SUBJECT_ID,Sepsis by ICD
87,164853,117,True
144,138376,124,True
276,172056,64,True
450,112077,85,True
747,111970,21,True
914,185910,38,True
1084,189535,61,True
1097,116009,62,True
1524,124271,250,True
1825,136614,191,True


# Part 2: Load Notes Data
Load the notes data, used for machine learning

In [62]:
# Load Discharge summaries
Discharges = Notes[Notes['CATEGORY']=="Discharge summary"]
Discharges = Discharges[['HADM_ID', 'CHARTDATE', 'TEXT']]
Discharges["Sepsis by Notes"] = [bool(i%2) for i in range(len(Discharges))]
Discharges.head()

,HADM_ID,CHARTDATE,TEXT,Sepsis by Notes
0,167853.0,2151-08-04,Admission Date: [**2151-7-16**] Dischar...,False
1,107527.0,2118-06-14,Admission Date: [**2118-6-2**] Discharg...,True
2,167118.0,2119-05-25,Admission Date: [**2119-5-4**] D...,False
3,196489.0,2124-08-18,Admission Date: [**2124-7-21**] ...,True
4,135453.0,2162-03-25,Admission Date: [**2162-3-3**] D...,False


# Part 3: Lab Values
Here we filter lab values to find patients likely to have sepsis. This will serve as a tiebreaker if the conclusion from the billing codes and the text do not match. We used the following criteria:
* Patient had at least one blood test that had a positive result
* Patient underwent antibiotic treatment that continued until at least 2 days after admission

In [63]:
# Load micro data and filter for positive blood tests
print(f"Original columns for Micro: {Micro.columns}\n")
BloodCx = Micro[Micro["SPEC_TYPE_DESC"]=="BLOOD CULTURE"]
BloodCx = BloodCx[~BloodCx["INTERPRETATION"].isna()]
BloodCx = BloodCx[['HADM_ID', 'CHARTDATE', 'CHARTTIME', 'INTERPRETATION']]
print(BloodCx.head(3))
print('\n'+'-'*45+'\n\n')

# Load Admissions data
Admissions = Admissions[['SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME', 'DEATHTIME', 'INSURANCE', 'ETHNICITY', 'DIAGNOSIS']]
print(Admissions.head(3))
print('\n'+'-'*45+'\n\n')

# Load Antibiotic data
Abx = pd.read_csv('Abx.csv')
Abx = Abx[['HADM_ID', 'ENDDATE', 'DRUG']]
print(Abx.head(3))
print('\n'+'-'*45+'\n\n')

Original columns for Micro: Index(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'CHARTDATE', 'CHARTTIME',
       'SPEC_ITEMID', 'SPEC_TYPE_DESC', 'ORG_ITEMID', 'ORG_NAME',
       'ISOLATE_NUM', 'AB_ITEMID', 'AB_NAME', 'DILUTION_TEXT',
       'DILUTION_COMPARISON', 'DILUTION_VALUE', 'INTERPRETATION'],
      dtype='object')

    HADM_ID            CHARTDATE            CHARTTIME INTERPRETATION
73   193281  2140-04-28 00:00:00  2140-04-28 14:45:00              S
74   193281  2140-04-28 00:00:00  2140-04-28 14:45:00              S
75   193281  2140-04-28 00:00:00  2140-04-28 14:45:00              R

---------------------------------------------


   SUBJECT_ID  HADM_ID            ADMITTIME            DISCHTIME DEATHTIME  \
0          22   165315  2196-04-09 12:26:00  2196-04-10 15:54:00       NaN   
1          23   152223  2153-09-03 07:15:00  2153-09-08 19:10:00       NaN   
2          23   124321  2157-10-18 19:34:00  2157-10-25 14:00:00       NaN   

  INSURANCE ETHNICITY                          

/tmp/ipykernel_113051/2483102082.py:15: DtypeWarning: Columns (11,16) have mixed types. Specify dtype option on import or set low_memory=False.
  Abx = pd.read_csv('Abx.csv')


In [64]:
# Merge data
BloodCxAdmin = pd.merge(BloodCx, Admissions, on="HADM_ID", how="inner")
SepsisTx = pd.merge(Abx, BloodCxAdmin, on="HADM_ID", how="inner")

# Filter antibiotics and drop duplicates
SepsisTx['ABXRXDAY'] = (pd.to_datetime(SepsisTx['ENDDATE']) - pd.to_datetime(SepsisTx['ADMITTIME'])).dt.days
SepsisTx=SepsisTx[SepsisTx['ABXRXDAY'] >=2]
SepsisTx = SepsisTx.drop_duplicates('HADM_ID')
SepsisTx = SepsisTx[['HADM_ID', 'INSURANCE', 'ETHNICITY', 'DIAGNOSIS']]
SepsisTx['Sepsis by Labs'] = True

SepsisTx.head()

,HADM_ID,INSURANCE,ETHNICITY,DIAGNOSIS,Sepsis by Labs
4,197487,Medicare,WHITE,AORTIC STENOSIS\AORTIC VALVE / ASCENDING AORTA...,True
84,108084,Medicare,UNKNOWN/NOT SPECIFIED,GASTROINTESTINAL BLEEDING,True
148,154851,Medicare,WHITE,BLADDER CANCER/SDA,True
420,157686,Medicare,WHITE,NAUSEA;VOMITING,True
720,148350,Medicare,WHITE,INTRAPARENCHYMAL HEMORRHAGE,True


# Part 4: Demographic Data
Loading demographic data

In [65]:
Patients = Patients[['SUBJECT_ID', 'GENDER']]
Patients.head()

,SUBJECT_ID,GENDER
0,249,F
1,250,F
2,251,M
3,252,M
4,253,F


# Part 5: Combining Data
From this point on, all the datasets will be combined into one

In [70]:
merged_data = pd.merge(sepsis_ICD, Discharges, on='HADM_ID', how='inner')
merged_data = pd.merge(merged_data, SepsisTx, on='HADM_ID', how='inner')
Sepsis = pd.merge(merged_data, Patients, on='SUBJECT_ID', how='left')
Sepsis
# merged_data

,HADM_ID,SUBJECT_ID,Sepsis by ICD,CHARTDATE,TEXT,Sepsis by Notes,INSURANCE,ETHNICITY,DIAGNOSIS,Sepsis by Labs,GENDER
0,138376,124,True,2166-02-01,Unit No: [**Numeric Identifier 13036**]\nAdmi...,True,Medicare,WHITE,PNEUMONIA;CHRONIC OBST PULM DISEASE,True,M
1,111970,21,True,2135-02-08,Admission Date: [**2135-1-30**] ...,False,Medicare,WHITE,SEPSIS,True,M
2,136614,191,True,2196-04-21,Admission Date: [**2196-4-9**] D...,True,Medicare,ASIAN - CHINESE,MYOCARDIAL INFARCTION,True,M
3,105694,223,True,2157-07-01,Admission Date: [**2157-5-1**] D...,False,Medicare,WHITE,PNEUMONIA-NSTEMI,True,M
4,106296,269,True,2170-11-27,Admission Date: [**2170-11-5**] ...,False,Medicaid,WHITE,SEPSIS;PILONIDAL ABSCESS,True,M
...,...,...,...,...,...,...,...,...,...,...,...
1432,164631,97263,True,2122-01-20,Admission Date: [**2122-1-15**] ...,True,Private,PATIENT DECLINED TO ANSWER,PNEUMONIA; AMS; THROMBOCYTOPENIA,True,M
1433,176834,99830,True,2187-09-14,Admission Date: [**2187-8-20**] ...,False,Medicaid,UNKNOWN/NOT SPECIFIED,S/P GUN SHOT WOUND,True,M
1434,101018,99836,True,2116-07-17,Admission Date: [**2116-5-21**] ...,False,Medicare,UNABLE TO OBTAIN,HEPATITIS,True,F
1435,102063,96958,True,2131-04-20,Admission Date: [**2131-3-22**] ...,False,Private,WHITE,ACUTE RENAL FAILURE,True,M


In [78]:
%pip install tableone
import tableone
tableone.tableone(
    Sepsis,
    columns= [
        'INSURANCE',
        'ETHNICITY',
        'DIAGNOSIS',
        'GENDER',
        'Sepsis by Labs',
        'Sepsis by Notes',
        'Sepsis by ICD',
    ]
)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 94.8 MB/s eta 0:00:00:00:010:01
  Using cached patsy-0.5.6-py2.py3-none-any.whl (233 kB)
Note: you may need to restart the kernel to use updated packages.


Missing       Overall
n                                                                                                                                                                                       1437
INSURANCE, n (%)       Government                                                                                                                                            0      26 (1.8)
                       Medicaid                                                                                                                                                    126 (8.8)
                       Medicare                                                                                                                                                   867 (60.3)
                       Private                                                                                                                                                    412 (28.7)
                       Self Pay                                                                                                                                                      6 (0.4)
ETHNICITY, n (%)       ASIAN                                                                                                                                                 0      28 (1.9)
                       ASIAN - ASIAN INDIAN                                                                                                                                         12 (0.8)
                       ASIAN - CAMBODIAN                                                                                                                                             1 (0.1)
                       ASIAN - CHINESE                                                                                                                                              12 (0.8)
                       BLACK/AFRICAN                                                                                                                                                 3 (0.2)
                       BLACK/AFRICAN AMERICAN                                                                                                                                     155 (10.8)
                       BLACK/CAPE VERDEAN                                                                                                                                            2 (0.1)
                       BLACK/HAITIAN                                                                                                                                                 2 (0.1)
                       CARIBBEAN ISLAND                                                                                                                                              2 (0.1)
                       HISPANIC OR LATINO                                                                                                                                           47 (3.3)
                       HISPANIC/LATINO - COLOMBIAN                                                                                                                                   1 (0.1)
                       HISPANIC/LATINO - CUBAN                                                                                                                                       3 (0.2)
                       HISPANIC/LATINO - DOMINICAN                                                                                                                                   1 (0.1)
                       HISPANIC/LATINO - MEXICAN                                                                                                                                     1 (0.1)
                       HISPANIC/LATINO - PUERTO RICAN                                                                                                                               10 (0.7)
         

In [79]:
tableone.tableone(
    Sepsis,
    columns= [
        'INSURANCE',
        'ETHNICITY',
        'DIAGNOSIS',
        'GENDER',
        'Sepsis by Labs',
        'Sepsis by Notes',
        'Sepsis by ICD',
    ],
    groupby=[
        'ETHNICITY',
    ]
)

Grouped by ETHNICITY                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
                                                                                                                                                                                    Missing       Overall       ASIAN ASIAN - ASIAN INDIAN ASIAN - CAMBODIAN ASIAN - CHINESE BLACK/AFRICAN BLACK/AFRICAN AMERICAN BLACK/CAPE VERDEAN BLACK/HAITIAN CARIBBEAN ISLAND HISPANIC OR LATINO HISPANIC/LATINO - COLOMBIAN HISPANIC/LATINO - CUBAN HISPANIC/LATINO - DOMINICAN HISPANIC/LATINO - MEXICAN HISPANIC/LATINO - PUERTO RICAN MIDDLE EASTERN MULTI RACE ETHNICITY       OTHER PATIENT DECLINED TO ANSWER PORTUGUESE UNABLE TO OBTAIN UNKNOWN/NOT SPECIFIED         WHITE WHITE - EASTERN EUROPEAN WHITE - RUSSIAN
n                                                                                                                                                                                                    1437          28                   12                 1              12             3                    155                  2             2                2                 47                           1                       3                           1                         1                             10              2                    4          34                          8          5               26                    56          1019                        1               2
INSURANCE, n (%)       Government                                                                                                                                                         0      26 (1.8)    3 (10.7)             3 (25.0)                                                                1 (0.6)                                                              3 (6.4)                                                                                                                                 1 (10.0)                                                                                                1 (3.8)                            14 (1.4)                                         
                       Medicaid                                                                                                                                                                 126 (8.8)    4 (14.3)              1 (8.3)         1 (100.0)        2 (16.7)      1 (33.3)              21 (13.5)           1 (50.0)      1 (50.0)                           11 (23.4)                   1 (100.0)                                                                     1 (100.0)                       1 (10.0)       1 (50.0)             1 (25.0)    7 (20.6)                              1 (20.0)          1 (3.8)             11 (19.6)      57 (5.6)                1 (100.0)                
                       Medicare                                                                                                                                                                867 (60.3)   17 (60.7)             5 (41.7)                          9 (75.0)                           110 (71.0)           1 (50.0)      1 (50.0)        2 (100.0)          22 (46.8)                                           3 (100.0)                                                                             7 (70.0)                                       14 (41.2)                   3 (37.5)                   18 (69.2)             28 (50.0)    625 (61.3)                                2 (100.0)
               